In [7]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords

dataset_fake = pd.read_csv("sample_data/Fake.csv")
dataset_fake["label"]=1
dataset_true = pd.read_csv("sample_data/True.csv")
dataset_true["label"]=0

dataset = pd.concat((dataset_fake,dataset_true), axis=0)
dataset = dataset.sample(frac=1).reset_index(drop=True)

X = dataset.title
y = dataset.label

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(X)):
    news = re.sub( "[^a-zA-Z]"," ",X[i])
    news = news.lower()
    news = news.split()
    sub_corpus = [ps.stem(word) for word in news if not word in stopwords.words("english")]
    sub_corpus = " ".join(sub_corpus)
    corpus.append(sub_corpus)

In [11]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

input_len = 1000
oh = [one_hot(word, input_len) for word in corpus]

ohp = pad_sequences(oh, padding = "post")
sent_len = len(ohp[0])

In [16]:
output_len = 50
model = Sequential()
model.add(Embedding(input_len, output_len, input_length=sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 50)            50000     
_________________________________________________________________
dropout (Dropout)            (None, 35, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 110,501
Trainable params: 110,501
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
X_final = np.array(ohp)
y_final = np.array(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size =0.2, random_state=20)


In [19]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=10, batch_size=64)
y_pred = model.predict_classes(X_test)

Epoch 1/10
562/562 [==============================] - 4s 7ms/step - loss: 0.0714 - accuracy: 0.9733 - val_loss: 0.3001 - val_accuracy: 0.9166
Epoch 2/10
562/562 [==============================] - 4s 7ms/step - loss: 0.0668 - accuracy: 0.9748 - val_loss: 0.2919 - val_accuracy: 0.9164
Epoch 3/10
562/562 [==============================] - 4s 7ms/step - loss: 0.0658 - accuracy: 0.9755 - val_loss: 0.2943 - val_accuracy: 0.9192
Epoch 4/10
562/562 [==============================] - 4s 7ms/step - loss: 0.0644 - accuracy: 0.9764 - val_loss: 0.2927 - val_accuracy: 0.9161
Epoch 5/10
562/562 [==============================] - 4s 8ms/step - loss: 0.0626 - accuracy: 0.9768 - val_loss: 0.2947 - val_accuracy: 0.9188
Epoch 6/10
562/562 [==============================] - 4s 7ms/step - loss: 0.0630 - accuracy: 0.9765 - val_loss: 0.2845 - val_accuracy: 0.9188
Epoch 7/10
562/562 [==============================] - 4s 7ms/step - loss: 0.0582 - accuracy: 0.9779 - val_loss: 0.3236 - val_accuracy: 0.9195
Epoch 

0.9204899777282851

In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

0.9204899777282851

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)